In [1]:
%cd ..

import pandas as pd
import os

from scripts.utils import *

pd.set_option('display.max_columns', None)


/run/media/nazif/2F946E411BA61D49/thesis


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl60/Homo_sapiens.GRCh37.60.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl60/Homo_sapiens.GRCh37.60.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl60/Homo_sapiens.GRCh37.60.pep.all.fa.gz.pickle


# import CLASH and make predictions

In [5]:
clash_df = pd.read_csv("data/processed/clash/clash_parsed.csv")
# df = find_matches_with_rnaduplex(clash_df)
# df.to_csv("results/1_checkpoint.csv", index=False)
df = pd.read_csv("results/1_checkpoint.csv")

# augmenting results df with data from CLASH

In [6]:
# identifiers
df["id"] = clash_df["id"]
df["mirna_sequence"] = clash_df["mirna_sequence"]
df["mirna_accession"] = clash_df["mirna_accession"]
df["enst"] = clash_df["enst"]

# full seq
df["full_sequence_of_transcript"] = clash_df["full_sequence_of_transcript"]

# extended sequence columns
df["extended_mrna_sequence"] = clash_df["extended_sequence"]
df["extended_mrna_start"] = clash_df["extended_start"]
df["extended_mrna_end"] = clash_df["extended_end"]

# non-extended sequence columns
df["clash_mrna_sequence"] = clash_df["mrna_sequence"]
df["clash_mrna_start"] = clash_df["true_start_index"]
df["clash_mrna_end"] = clash_df["true_end_index"]

# true values
df["true_seed_type"] = clash_df["true_seed_type"]
df["true_num_basepairs"] = clash_df["num_basepairs"]
df["true_seed_basepairs"] = clash_df["seed_basepairs"]
df["true_energy"] = clash_df["folding_energy"]

# generating features

In [8]:
# feature generation
df = generate_alignment_string_from_dot_bracket(df)
df = generate_match_count_columns(df)

# adding bp difference columns
df["bp_difference"] = df["pred_num_basepairs"] - df["true_num_basepairs"]
df["seed_bp_difference"] = df["pred_seed_basepairs"] - df["true_seed_basepairs"]

# getting mirna length
df["mirna_length"] = df["mirna_sequence"].str.len()

# using mirna length to figure out mre coordinates
df["mre_end"] = df["clash_mrna_start"] + df["mrna_end"] + df["mirna_start"]
df["mre_start"] = df["mre_end"] - df["mirna_length"]

# some start values are lower than zero, so we need to adjust
df['mre_start'] = df['mre_start'].clip(lower=0)

# creating mre sequence column
df['mre_region'] = df.apply(lambda row: row['full_sequence_of_transcript'][row['mre_start']:row['mre_end']], axis=1)


df.head()

,mrna_start,mrna_end,pred_energy,mirna_start,mirna_end,mirna_dot_bracket_5to3,id,mirna_sequence,mirna_accession,enst,full_sequence_of_transcript,extended_mrna_sequence,extended_mrna_start,extended_mrna_end,clash_mrna_sequence,clash_mrna_start,clash_mrna_end,true_seed_type,true_num_basepairs,true_seed_basepairs,true_energy,alignment_string,pred_num_basepairs,pred_seed_basepairs,bp_difference,seed_bp_difference,mirna_length,mre_end,mre_start,mre_region
0,22,45,-27.7,0,20,)))))))))))))))))))).,0727A-1038930_1,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000340828,AGGGCAGGCGCGGCCCCTTCGGCTCCGAGCTGACCCTGATCAGGGC...,TTTAATATTTTTTTCTAGAAAACAGGTGACATTTGTATCTACGATA...,1760,1920,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,1790,1890,noncanonical_seed,20,6,-25.1,1111111111111111111100,20,6,0,0,22,1835,1813,TTATACAGAACCTACTGCCTCA
1,11,30,-26.2,0,18,))))))))).)))))))).,L1HS-1112536_1,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000343455,CGGAGGCGCGGCGCAGGCTGCTGCAGGCCCAGGTGAATGGAGTAAC...,TGCCAAGGAAATCAGCTAAATTACTACAAGCAGGAAATACCCGTGC...,3826,3958,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,3856,3928,9-mer,17,6,-24.4,1111111110111111110000,17,6,0,0,22,3886,3864,ACCCGTGCAACCAACTACCTCA
2,3,24,-23.6,0,21,.)))))).))))))))))))).,L2HS-818542_2,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000436639,GATTGCCAGGGCCGCCCTGTGCCCTCTGGCTCGGCGGTGGTGGGCG...,GCAGAACTCCTTTATGCTCTGAGAGCCATTACCCGCTATATGACCT...,2354,2464,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,2384,2434,noncanonical_seed,19,6,-22.2,0111111011111111111110,19,6,0,0,22,2408,2386,CCGCTATATGACCTGATGCCTT
3,3,24,-23.8,0,21,)))))))...)))))))))))),L2HS-1161339_2,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000282516,TCCGGTCGGCATTTTGTTCTGAGAGGGAGAGACGGAACGAGAGAGA...,AAATAAGACCCCAGCTCATGGTTAAACATGCAATGACTATGCAACC...,6539,6653,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,6569,6623,noncanonical_seed,18,6,-22.1,1111111000111111111111,19,6,1,0,22,6593,6571,ATGACTATGCAACCATACCTTA
4,2,24,-24.0,0,21,))))))))))))))...)))).,L2-407944_2,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000340139,AGGGACCACCGGGAACAGACGGATCGGCAGGGCGGGGCGGAACGGT...,ACAGCAAAATTGAGTTCAACAACACAAAACAATACTGGGAAACCTG...,1133,1238,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,1163,1208,noncanonical_seed,18,6,-21.9,1111111111111100011110,18,6,0,0,22,1187,1165,TACTGGGAAACCTGCTACTTCG


In [9]:
# writing to csv
df.to_csv("results/1_find_matches_for_clash_data.csv", index=False)